In [1]:
import os
import pdb
import datetime

import numpy as np
import pandas as pd


import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt


#import plottools as pt
import remtools as rt
import scoreblock as sb
import eegplotter as ep


%load_ext autoreload

In [2]:

def dig_out_scores(s=None, trial=None, classifier='OVR', m='24h_8ch'):
    """quick hack to get a human and model scores for one trial"""

    # WWRW scoreTag and scoreType columns, then the vector of scores
    # scoreType (human, model, consensus, switch, etc)
    # scoreTag (h-con, h-GS, m-OVR-24h_8ch, etc)
    
    
    
    # narrow down to trial
    scbt = scb.keeprows(conditions=[('trial', trial)])

    # human scores
    df_keep = scbt.df[~scbt.df['scorer'].isna()].copy()
    scb_hum = sb.ScoreBlock(df=df_keep, index_cols=scbt.copy_index_cols())

    scoreType = ['human']*scb_hum.numrows
    scoreTag = [ 'hum-%s' % (row['scorer']) for _, row in scb_hum.df.iterrows()]
    scb_hum.df['scoreType'] = scoreType
    scb_hum.df['scoreTag'] = scoreTag
    scb_hum.index_cols += ['scoreType', 'scoreTag']

    
    # model scores
    cc = [('classifier', classifier), ('M', m)]
    scb_mdl = scbt.keeprows(conditions=cc, comparison='all')

    scoreType = ['model']*scb_mdl.numrows
    scoreTag = [ '%s-%s' % (row['classifier'], row['M']) for _, row in scb_mdl.df.iterrows()]
    scb_mdl.df['scoreType'] = scoreType
    scb_mdl.df['scoreTag'] = scoreTag
    scb_mdl.index_cols += ['scoreType', 'scoreTag']



    scb_out = scb_hum.stack(others=[scb_mdl])

    return scb_out

In [3]:

f_pca = '../../sandbox/ANL-pca-A/pca-test.json'
#f_pcaproj = '../../sandbox/ANL-pca-A/df_pca_prj.csv'
f_features = '../../sandbox/ANL-preprocess-A-train/trial-374/staged-trial-data.json'
f_scores = '../../sandbox/ANL-merged-scores-A/scoreblock-raw-merged.json'


#f_features = '../../sandbox/ANL-preprocess-B-blind/trial-2391/staged-trial-data.json'
#f_scores = '../../sandbox/ANL-merged-scores-B/scoreblock-raw-merged.json'


In [4]:
#%autoreload 2

#edf = rt.EDFData(f_edf)
#pcaprj = pd.read_csv(f_pcaproj, index_col=0)
std = rt.StagedTrialData.from_json(f_features)
pca = rt.PCA.from_json(f_pca)
scb = sb.ScoreBlock.from_json(f_scores)

sbstack = dig_out_scores(s=scb, trial=std.trial, classifier='OVO', m='24h_8ch')

STAGING PARAMETERS SHOULD BE DEPRECATED
-----------------------------------------------------------------------
file               : ../../sandbox/ANL-preprocess-A-train/trial-374/../../../../data/raw_edf/400hz/374.edf
trial              : 374
num datarecords    : 8640
num annotations    : 0
number of signals  : 4
duration [s]       : 86400
startdatetime      : 2019-01-02 9:00:00
epoch duration [s] : 10.0
samples in file    : [34560000 34560000 34560000  3456000]
samples_per_epoch  : [4000. 4000. 4000.  400.]
frequency [Hz]     : [400. 400. 400.  40.]
signal_labels      : ['EEG1', 'EEG2', 'EMG', 'EDFAnnotations']
------------------------------------
  loading signal trace: EEG1
  loading signal trace: EEG2
  loading signal trace: EMG
  loading signal trace: EDFAnnotations
  computing spectrogram: EEG1
  computing spectrogram: EEG2
  computing spectrogram: EMG
  edf load complete: ../../sandbox/ANL-preprocess-A-train/trial-374/../../../../data/raw_edf/400hz/374.edf
---------------------

/home/gbubnis/anaconda2/envs/mouse1/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3209: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



In [5]:
%autoreload 2
#matplotlib.rcdefaults()

pp = ep.EEGPlotter(std=std, pca=pca, scores=sbstack)
pp.render(viewEpoch=500)

In [6]:
%autoreload 2
#%matplotlib inline

matplotlib.rcdefaults()

class trace_dumper(object):
    """visualize a 24h recording and accompanying scores
    
    This will fit 1 hour of data per page using 6 rows/page and 60 epochs/row
    
    
    """
    def __init__(self, std=None, scores=None):
        """
        
        input
        ------
        std (StagedTrialData) : data class
        scores (ScoreBlock) : block of scores (human,model etc)

        """
        
        import itertools

        self.std = std
        self.scores = scores
        

        # hard codey stuff
        self.epoch_length = 10          # 10s
        self.epochs_per_panel = 60     # 180 = 30 min
        self.panels_per_page = 6
        signal_names = ['EEG1', 'EEG2', 'EMG']

        
        # stage data traces, precompute strides etc ranges
        self.signals = {}
        for name in signal_names:
            trace = std.edf.signal_traces[name]

            epoch_stride = int(trace.samples_per_epoch)
            panel_stride = int(trace.samples_per_epoch*self.epochs_per_panel)
            page_stride = int(panel_stride*self.panels_per_page)

            num_pages = int(np.ceil(len(trace.sig)/page_stride))

            #print('-------------------------')
            #print('trace       :', name)
            #print('len trace   :', len(trace.sig))
            #print('panel_stride:', panel_stride)
            #print('page_stride :', page_stride)
            #print('num_pages   :', num_pages)

            pp = itertools.product(range(num_pages), range(self.panels_per_page))
            df = pd.DataFrame(data=pp, columns=['page', 'panel'])
            df['ia'] = df['page']*page_stride + df['panel']*panel_stride
            df['ib'] = df['ia'] + panel_stride

            sig = trace.sig/np.std(trace.sig)
            
            
            dd = dict(
                name=name,
                df_index=df,
                tvec=trace.tvec,
                sig=sig,
                epoch_stride=epoch_stride,
                panel_stride=panel_stride,
                page_stride=page_stride,
                num_pages=num_pages
            )

            self.signals[name] = dd
        
        
        #print(self.std.edf.metadata)
        self.num_pages = np.unique([x['num_pages'] for x in self.signals.values()])
        if len(self.num_pages) > 1:
            raise Exception('num_pages conflict')
        else:
            self.num_pages = self.num_pages[0]
        
        
        # panel/epoch indexing stuff
        # epoch time series
        pp = itertools.product(range(num_pages), range(self.panels_per_page))
        df = pd.DataFrame(data=pp, columns=['page', 'panel'])
        
        df['ta'] = (df['page']*self.panels_per_page + df['panel'])*self.epochs_per_panel*self.epoch_length
        df['tb'] = df['ta']+self.epochs_per_panel*self.epoch_length
        
        #df['ea'] = df['ta']/self.epoch_length+1
        #df['eb'] = df['ea']+self.epochs_per_panel
        #print(df.head(20))
        
        self.df_panel_index = df
        
        self.make_fig()
        

    def make_fig(self):
        """create the figure"""

        # nicer default colors
        aa = '707070'
        #aa = 'a0a0a0'
        b0 = 'b0b0b0'
        gg = 'c0c0c0'
        
        # black background
        pp = {
            'lines.linewidth':2,
            'axes.facecolor':'k',
            'axes.edgecolor': gg,
            'axes.labelcolor': gg,
            'figure.facecolor':'k',
            'figure.edgecolor':'k',
            'savefig.facecolor':'k',
            'savefig.edgecolor':'k',
            'xtick.color' : gg,
            'ytick.color' : gg,
            'grid.color'  : aa,
            'text.color' : gg
        }
        
        # white bg w/gray lines/text
        # good for printing
        pp = {
            'lines.linewidth':2,
            'axes.edgecolor': aa,
            'axes.labelcolor': aa,
            'xtick.color' : aa,
            'ytick.color' : aa,
            'grid.color'  : aa,
            'text.color' : aa
        }

        matplotlib.rcParams.update(pp)
        
        self.fig = plt.figure(figsize=(24, 16), dpi=60)

    def export(self, filename='trace-dump.png'):
        """"""
        self.fig.savefig(filename)
        
        
    def render_page(self, page=0):
        """"""
        


        scoremap = {
            'Wake':0,
            'REM':0.5,
            'Non REM':0.999,
            'Non REM X':np.nan,
            'REM X':np.nan,
            'Unscored':np.nan,
            'Wake X':np.nan,
            'XXX':np.nan
        }

        epoch_tick_stride = 6
        stride = 1
        
        dy_neg = +7
        dy_pos = -2

        plot_signal_kwa = dict(lw=1, color='grey', alpha=0.7)
        
        # nuke what already exists
        self.fig.clf()        
        ax = [plt.subplot(self.panels_per_page, 1, i+1) for i in range(self.panels_per_page)]

        # one panel at a time
        for panel in range(self.panels_per_page):
            # time series plotted with negative dy shifts
            # categorical/imshow plotted with pos dy shifts
            
            # PANEL STUFF
            dfp = self.df_panel_index.copy()
            dfp =  dfp[(dfp['page'] == page) & (dfp['panel'] == panel)]
            
            # time limits
            [ta, tb] = dfp[['ta','tb']].values.ravel()

            # epoch limits
            ea = int(ta/self.epoch_length)
            eb = int(ea+self.epochs_per_panel)
            
            # RAW SIGNALS
            for iy, name in enumerate(['EMG', 'EEG1', 'EEG2']):

                dyi = (iy+1)*dy_neg

                sig = self.signals[name]
                
                df = sig['df_index']
                df = df[(df['page'] == page) & (df['panel'] == panel)]
                [ia, ib] = df[['ia','ib']].values.ravel()

                ax[panel].plot(sig['tvec'][ia:ib:stride], sig['sig'][ia:ib:stride]+dyi, **plot_signal_kwa)
                ax[panel].text(tb+1, dyi, name, ha='left', va='center')
                
                #print('sig/panel/ia/ib:', name, panel, ia, ib)
                #print('num_pts:', ib-ia)

                ylim_max = (iy+1+0.5)*dy_neg

            # SCORES (categorical)
            scoreTypes = self.scores.df['scoreType'].unique()
            scoreTags = self.scores.df['scoreTag'].unique()

            for j, scoreTag in enumerate(scoreTags):

                data = self.scores.keeprows(conditions=[('scoreTag', scoreTag)])
                # print(scoreTag)
                # print(data.data)

                dyj = dy_pos*(0.5+j)
                ylim_min = dy_pos*(1+j)

                ax[panel].text(tb+1, dyj, scoreTag, ha='left', va='center')

                dfj = data.data.ravel()[ea:eb]
                tvals = np.linspace(ea, eb-1, self.epochs_per_panel)*self.epoch_length
                tvals += self.epoch_length/2
                
                wk = tvals[dfj == 'Wake']
                rm = tvals[dfj == 'REM']
                nr = tvals[dfj == 'Non REM']
                
                kwa = dict(ms = 5, alpha=0.3, lw=0)
                if j == len(scoreTags)-1:
                    ax[panel].plot(wk, 0*wk+dyj, marker=r'$\mathtt{W}$', color='blue',  label='Wake', **kwa)
                    ax[panel].plot(rm, 0*rm+dyj, marker=r'$\mathtt{R}$', color='red',   label='REM',  **kwa)
                    ax[panel].plot(nr, 0*nr+dyj, marker=r'$\mathtt{N}$', color='green', label='Non REM', **kwa)


            
            # numerical score values for the current inteval
            scnum = self.scores.mask(mask=slice(ea, eb)).applymap(scoremap)
            
            extent = (ta, tb, 0, ylim_min)
            #ax[panel].imshow(scnum.data, origin='lower', aspect='auto', extent=extent, alpha=0.5)
            #ax[panel].imshow(scnum.data, origin='lower', cmap='tab10', aspect='auto', extent=extent)
            ax[panel].imshow(scnum.data, origin='lower', cmap='plasma', aspect='auto', extent=extent, vmin=0, vmax=1)
            
            if page == 2 and panel == 1:
                
                print('epoch 804 renders R but imshow color is wrong')
                
                #pdb.set_trace()

            # xticks
            numticks = int(self.epochs_per_panel/epoch_tick_stride)+1
            xtk = np.linspace(ta, tb, numticks)
            ax[panel].set_xticks(xtk)

            # fancy stuff
            ax[panel].set_yticks([])
            ax[panel].spines['top'].set_visible(False)
            ax[panel].spines['bottom'].set_visible(False)
            ax[panel].spines['left'].set_visible(False)
            ax[panel].spines['right'].set_visible(False)
            ax[panel].grid()
            ax[panel].set_xlim([ta, tb])
            
            ax[panel].set_ylim([ylim_min, ylim_max])
 
            ax[panel].axhline(y=0, lw=1, color='grey')

    
        # global tidying, annotation
        plt.tight_layout(h_pad=4)
        plt.subplots_adjust(top=0.95)

        txt = datetime.datetime.now().replace(microsecond=0).isoformat().replace('T','\n')
        tx1 = self.fig.text(0.01, 0.99, txt, ha='left', va='top', fontsize=20)

        txt = 'trial: %i  day: %i  page: %i/%i' % (self.std.trial, self.std.tagDict['day'], page+1, self.num_pages)
        tx2 = self.fig.text(0.5, 0.99, txt, ha='center', va='top', fontsize=42)

        self.fig.show()



In [8]:
td = trace_dumper(std=std, scores=sbstack)

#print(td.num_pages)
td.render_page(0)


for page in range(24):
    td.render_page(page)
    td.export('page-%2.2i.png' % (page))

epoch 804 renders R but imshow color is wrong
